In [24]:
import torch
import cv2
import numpy as np
import pandas as pd
import torchvision as tv
import ipywidgets as widgets
import matplotlib.pyplot as plt
from PIL import Image
from ipywidgets import interact, interactive

In [31]:
DATA_PATH = '/home/ubuntu/data/dogbreed'
labels_df = pd.read_csv(f'{DATA_PATH}/labels.csv')
labels_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [ ]:
# - (1) Rescale images to have shortest side of 340px.
# - (2) Take center 224 x 224 crop.

In [ ]:
source_dir = '/home/ubuntu/data/dogbreed/train'
dest_dir = '/home/ubuntu/data/dogbreed/tmp/funk/340'
img_names = labels_df['id'].map(lambda id: id + '.jpg')
transform_files(img_names, source_dir, dest_dir, partial(resize_image, 340))

In [45]:
def show_input_image(idx):
    img = pil_tform(Image.open(f'{dest_dir}/{img_names.iloc[idx]}'))
    plt.imshow(img)

img_names = labels_df['id'].map(lambda id: id + '.jpg')
pil_tform = tv.transforms.Compose([
    tv.transforms.RandomRotation(10),
    tv.transforms.ColorJitter(0.05,0.05,0.05,0.05),
    tv.transforms.RandomHorizontalFlip(0.5),
    tv.transforms.CenterCrop(224)])
interact(show_input_image, idx=widgets.IntSlider(min=0, max=img_names.shape[0] -1 , value = 0))

interactive(children=(IntSlider(value=0, description='idx', max=10221), Output()), _dom_classes=('widget-inter…

<function __main__.show_input_image(idx)>

# Scratchpad while training in the other notebook

In [49]:
from resnext_101_64x4d import resnext_101_64x4d

In [52]:
model = resnext_101_64x4d()
model.load_state_dict(torch.load('/home/ubuntu/data/weights/resnext_101_64x4d.pth'))

In [60]:
def cond_init(m, init_fn):
    if not isinstance(m, (torch.nn.BatchNorm1d, torch.nn.BatchNorm2d, torch.nn.BatchNorm3d)):
        if hasattr(m, 'weight'): init_fn(m.weight)
        if hasattr(m, 'bias') and hasattr(m.bias, 'data'): m.bias.data.fill_(0.)

def apply_init(m, init_fn):
    m.apply(lambda x: cond_init(x, init_fn))

    
#apply_init(model, torch.nn.init.kaiming_normal)

#model.apply(lambda submodule: cond_init(submodule, torch.nn.init.kaiming_normal_))

#cond_init(model, )

In [ ]:
# ------
# TODO

# - Match fast.ai default performance.
# - Learning rate jazz
#   - Cyclical learning rates: https://arxiv.org/abs/1506.01186
#   - Setting the learning rate: https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0
#   - LR finder
# - The rest of JH's training tricks
# - Sanity - composable traning loop
# - TTA

# annoyances
# -- Speed - check against fast.ai, check you're not missing any optimisations
# -- Pytorch transforms seem slow; is handrolled opencv implementation faster?
# -- Store best weights as you go
# -- Load / Save weights